In [2]:
import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers,IQPEmbedding
from pennylane.operation import Tensor


In [ ]:
def circuit(feature_vector,feature_vector_2,length):
#    qml.AngleEmbedding(features=feature_vector, wires=range(length),rotation='Z')
#    qml.adjoint(qml.AngleEmbedding(features=feature_vector_2, wires=range(length),rotation='Z'))
    qml.IQPEmbedding(features=feature_vector, wires=range(length),n_repeats=2)
    qml.adjoint(qml.IQPEmbedding(features=feature_vector_2, wires=range(length),n_repeats=2))
    return qml.probs(wires=range(length))
n_qubits = len(X_train[0])
dev_kernel = qml.device("default.qubit", wires=n_qubits)

In [4]:

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    u_1 = qml.matrix(circuit)(x1,x2,len(x1))
    u_2 = u_1.conjugate().transpose()
    projector = u_1+u_2
    return qml.expval(qml.Hermitian(projector,wires=range(n_qubits)))

In [13]:
# Samples are in rows.
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[np.exp(-2 + kernel(a, b)) for b in B] for a in A])

In [14]:
def qsvm(Xtrain,ytrain):
    svm = SVC(kernel=kernel_matrix).fit(Xtrain, ytrain)
    return svm